In [43]:
import numpy as np
import pandas as pd

#### 1. 크롤링 한 파일들 통합하기

In [44]:
fname = '가 나 다 라 마 바-1 바 사 사-1 사-2 아 자 차 카 타 파 하 A B C D E F G H I J K L M N O P Q R S T U V W X Y Z'.split()

In [45]:
df = pd.DataFrame()
for fn in fname:
    try:
        tmp = pd.read_csv(f'../static/data/movie{fn}.csv')
    except:
        continue
    df = pd.concat([df, tmp])
df.shape

(11273, 10)

In [46]:
df.head(1)

,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis
0,20231523,그 여름 (The Summer),애니메이션(2),"장편 | 예술,독립 영화 | 애니메이션, 드라마, 멜로/로맨스 | 60분 57초 |...",2023-06-07,2023년,한지원,윤아영 | 송하림,https://www.kobis.or.kr/common/mast/movie/2023...,"""서로의 몸은 차라리 꽃잎과 물결에 가까웠다""\r\n갈색 눈동자를 가진 평범한 학생..."


In [47]:
# index 정리
df.set_index('code', inplace=True)
df.reset_index(inplace=True)
df.fillna('', inplace=True)
print(df.shape)
df.head(1)


(11273, 10)


,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis
0,20231523,그 여름 (The Summer),애니메이션(2),"장편 | 예술,독립 영화 | 애니메이션, 드라마, 멜로/로맨스 | 60분 57초 |...",2023-06-07,2023년,한지원,윤아영 | 송하림,https://www.kobis.or.kr/common/mast/movie/2023...,"""서로의 몸은 차라리 꽃잎과 물결에 가까웠다""\r\n갈색 눈동자를 가진 평범한 학생..."


In [48]:
df.isna().sum()

code               0
title              0
genre              0
summering          0
first_day          0
production_year    0
movie_director     0
star_actor         0
img                0
synopsis           0
dtype: int64

In [49]:
# 간혹 같은 영화가 들어가 있어 중복 삭제
df.drop_duplicates(subset='code', inplace=True)

In [50]:
df.code.nunique(), df.shape

(11271, (11271, 10))

In [51]:
# summering 적은 것 삭제
df[df.summering == ('장편 | 일반영화 | 공연')]

,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis
3428,20193151,마리아 스투아르다 (Maria Stuarda ),공연,장편 | 일반영화 | 공연,해당정보없음,2020년,데이비드 맥비커,,https://www.kobis.or.kr/kobis/web/comm/images/...,스코틀랜드의 메리 스튜어트 여왕과 잉글랜드의 엘리자베스 여왕. 권력과 사랑을 놓고 ...


In [52]:
df.drop(df.index[3428], inplace=True)
df.shape

(11270, 10)

In [53]:
# 다시 인덱스 정리
df.set_index('code', inplace=True)
df.reset_index(inplace=True)

In [54]:
# summering 따로따로 분리해서 추가
m_type, m_kind, m_genre, m_time, m_rated, m_nation = [], [], [], [], [], []
for info in df.summering.values:
    infos = info.split('|')
    infos_len = len(infos)
    m_type.append(infos[0].strip())                     # 장편
    m_kind.append(infos[1].strip())                     # 일반영화, 예술영화..
    m_genre.append(infos[2].strip().replace(',', ''))   # 애니메이션, 드라마, 멜로
    m_nation.append(infos[-1].strip())                  # 한국
    
    # 요약 정보 개수가 다를 수 있다
    if infos_len == 6:
        m_time.append(infos[-3].strip())                # 70분
        m_rated.append(infos[-2].strip())
    elif infos_len == 5 :
        m_time.append(infos[-2].strip())                # 12세 이상 관람가
        m_rated.append('')
    elif infos_len == 4:
        m_rated.append('')
        m_time.append('')
    else:
        print(info)

len(m_type)

11270

In [55]:
df['m_type'] = m_type
df['m_kind'] = m_kind
df['m_genre'] = m_genre
df['m_nation'] = m_nation
df['m_time'] = m_time
df['m_rated'] = m_rated

In [56]:
df.head(1)

,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis,m_type,m_kind,m_genre,m_nation,m_time,m_rated
0,20231523,그 여름 (The Summer),애니메이션(2),"장편 | 예술,독립 영화 | 애니메이션, 드라마, 멜로/로맨스 | 60분 57초 |...",2023-06-07,2023년,한지원,윤아영 | 송하림,https://www.kobis.or.kr/common/mast/movie/2023...,"""서로의 몸은 차라리 꽃잎과 물결에 가까웠다""\r\n갈색 눈동자를 가진 평범한 학생...",장편,"예술,독립 영화",애니메이션 드라마 멜로/로맨스,한국,60분 57초,12세이상관람가


#### 2. 시놉시스 형태소 분석해 놓기

In [57]:
from konlpy.tag import Okt
okt = Okt()
stop_words = '''의 가 에 과 을 의 를 는 이 은 와 
            '''.split()

In [58]:
import re, string

morphs = []

for text in df.synopsis:
    text = re.sub('['+string.punctuation+'\r\n“”‘’…]', ' ', text)
    tokens = okt.morphs(text)
    morphs.append(' '.join([word for word in tokens if word not in stop_words]))

df['morphs'] = morphs


In [61]:
df.head(1)

,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis,m_type,m_kind,m_genre,m_nation,m_time,m_rated,morphs
0,20231523,그 여름 (The Summer),애니메이션(2),"장편 | 예술,독립 영화 | 애니메이션, 드라마, 멜로/로맨스 | 60분 57초 |...",2023-06-07,2023년,한지원,윤아영 | 송하림,https://www.kobis.or.kr/common/mast/movie/2023...,"""서로의 몸은 차라리 꽃잎과 물결에 가까웠다""\r\n갈색 눈동자를 가진 평범한 학생...",장편,"예술,독립 영화",애니메이션 드라마 멜로/로맨스,한국,60분 57초,12세이상관람가,서로 몸 차라리 꽃잎 물결 가까웠다 갈색 눈동자 가진 평범한 학생 이경 여름 햇살 ...


#### 3. 최종 파일 저장

In [62]:
df.to_csv('../static/data/movie.csv', index=False)